In [1]:
# conda env : freecad_fem_env

import os
import sys

sys.path.append(os.path.join(os.environ["CONDA_PREFIX"], "lib"))

import FreeCAD
import Part
import ObjectsFem
import Mesh
import MeshPart
import Fem
import femmesh.femmesh2mesh
from femtools import ccxtools
import trimesh
import numpy as np
import time

from FreecadFemLib import *


In [2]:
#init
doc = App.newDocument("doc")

In [3]:
trimesh_scene_meshes = []
constraint_scene_meshes = []

In [4]:
doc,trimesh_scene_meshes = document_setup(doc,trimesh_scene_meshes)


 ******* Mesh.obj is loaded ******* 

mesh result mesh is removed...
shape result mesh is removed...


In [5]:
for state_indx in range(1,5):

    #------  setup the state-action
    print(f"\n\n-----------\n state - {state_indx}\n-----------\n\n")
    force_position = np.random.randint(3,70)
    print(f"force position : {force_position}")

    doc = create_shape_femmesh(doc)    
    doc , fixed_indx = set_constraint_fixed(doc)
    doc , force_indx = set_constraint_force_placement(doc, force_position=force_position)    
                                            
    if len(fixed_indx)==0 or len(force_indx)==0:
        print("force or fixed constraints are empty... no more analysis will be executed... ")
        break
        
    fixed_scene = add_constraints_to_scene_meshes(doc,fixed_indx,color='blue')
    force_scene = add_constraints_to_scene_meshes(doc,force_indx,color='red')
    constraint_scene_meshes.append(concat_meshes(fixed_scene,force_scene))

    doc = remove_old_mesh_result_mesh(doc)
    doc = remove_old_shape_result_mesh(doc)
    
    doc = set_constraint_force_value(doc, force=15000)
    # run Analysis
    doc, fem_volume = run_analysis(doc)
    if not fem_volume: break

    #------   create and export result mesh
    doc, out_mesh = create_mesh_from_result(doc)
    if not out_mesh: break

    result_trimesh = Meshobj_to_trimesh(out_mesh)
    result_trimesh_decimated = result_trimesh.simplify_quadratic_decimation(state_indx*2*1000)
    trimesh_decimated_topology = trimesh_to_mesh_topology(result_trimesh_decimated)
    result_trimesh_decimated = set_trimesh_color(result_trimesh_decimated)

    trimesh_scene_meshes.append(result_trimesh_decimated)
    #view_current_state(doc, result_trimesh_decimated)
    save_trimesh(result_trimesh_decimated,state_indx)
    export_result_mesh(doc,state_indx)
    
    #------   prepare shape for next state
    doc = remove_old_femmesh(doc)
    doc = clear_constraints(doc)
    doc = remove_old_solid(doc)

    doc = create_shape_from_mesh(doc,trimesh_decimated_topology)
    doc, solid = convert_to_solid(doc) 
    



-----------
 state - 1
-----------


force position : 26
num of faces as fixed constraints: 8
num of faces as force constraints: 2
Analysis done successfully...
CCX_Results
Mesh by surface search method:  0.4804850000000003
mesh_out is converted to mesh...
result mesh is exported as .obj file ... 
old femmesh is removed...
FemConstraints cleared...
old solid result mesh is removed...
shape from mesh done...


-----------
 state - 2
-----------


force position : 58
num of faces as fixed constraints: 16
num of faces as force constraints: 2
mesh result mesh is removed...
shape result mesh is removed...
Analysis done successfully...
CCX_Results
Mesh by surface search method:  0.8818970000000004
mesh_out is converted to mesh...
result mesh is exported as .obj file ... 
old femmesh is removed...
FemConstraints cleared...
old solid result mesh is removed...
shape from mesh done...


-----------
 state - 3
-----------


force position : 20
num of faces as fixed constraints: 32
num of faces 

In [6]:
mesh_scene = generate_complete_meshes_scene(doc,trimesh_scene_meshes)
mesh_scene.show()

In [7]:
print(len(constraint_scene_meshes),len(trimesh_scene_meshes))

n = np.random.randint(len(constraint_scene_meshes))
print(f"the [state{n} - action{n}] (before running the analysis)")
state_scene = []
state_scene.append(trimesh_scene_meshes[n])
state_scene.append(constraint_scene_meshes[n])

_scene = generate_complete_meshes_scene(doc,state_scene)
_scene.show()

4 5
the [state0 - action0] (before running the analysis)


In [8]:
s = scene_view_normals(result_trimesh_decimated)
s.show()

In [9]:
### save_image() has problem with opengl (and jupyter)

# from PIL import Image
# import io 
# image = np.array(Image.open(io.BytesIO(trimesh.Scene(pnt).save_image(resolution=(200, 200)))))

# trimesh.Scene(pnt).show(line_settings={'point_size':130})